<a href="https://colab.research.google.com/github/Echo9k/3-potential_talents/blob/dev/Potential%20Talent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Candidate Ranking and Re-Ranking with Starring

We will implement this in a two-step process:

1. **Initial Ranking**: Rank candidates based on their `fit` score.
2. **Re-Ranking with Starring**: Incorporate supervisory signals (starring) dynamically into a Learning-to-Rank (LTR) algorithm. We can use a pairwise ranking approach (e.g., LambdaMART).

## Introduction to Candidate Ranking
This section introduces the learning-to-rank approach used to evaluate candidates based on their "fit" score. The method enhances the ranking accuracy by incorporating real-time feedback, significantly reducing manual effort.

### Implementation Code

In [73]:
import os
import sys


try:
    from google.colab import drive
    drive.mount('/content/drive')
    root_dir = "/content/drive/MyDrive/wdir/repos/Apziva/3-potential_talents/"
    os.getcwd()

except ImportError:
    while 'potential_talents' not in os.listdir('.'):
        os.chdir('..')
        root_dir=os.getcwd()
    
    # append term_deposit to system to import custom functions
    sys.path.append('.')
    
%pwd

'/workspaces/3-potential_talents'

#### Dependencies

In [74]:
from pathlib import Path
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

#### Initial Setup

In [75]:
data_path = Path("data")
data = pd.read_parquet(data_path / "processed" / "grouped_results.parquet")\
    .sort_values('fit', ascending=False)
# Initial Ranking
data['rank'] = range(1, len(data) + 1)
# Initial Starred
data['is_starred'] = 0
data = data.reset_index()
data.head()

,job_title,fit,rank,is_starred
0,human resource staffing recruiting professional,0.748748,1,0
1,retired army national guard recruiter office m...,0.729468,2,0
2,aspiring human resource professional energetic...,0.725290,3,0
3,not tech seeking human resource payroll admini...,0.711246,4,0
4,experienced retail manager aspiring human reso...,0.709471,5,0


## Training the LambdaMART Model
- The **LambdaMART algorithm** is used for supervised learning-to-rank tasks.
- Features are engineered from candidate data to represent relevancy scores.
- Supervisory signals (e.g., recruiter "stars") are integrated to refine rankings dynamically.

In [76]:
def star_candidate(data):
    """
    Allow the user to interactively star a candidate and update the rankings.
    """
    print("\nCurrent Candidates:")
    print(data[['job_title', 'fit', 'rank']])

    # User input to star a candidate by job title or rank
    user_input = input("\nEnter the job title or rank of the candidate to star: ")
    
    try:
        # Check if input is a rank
        if user_input.isdigit():
            rank = int(user_input)
            if rank in data['rank'].values:
                data.loc[data['rank'] == rank, 'is_starred'] = 1
            else:
                print("Invalid rank. Please try again.")
                return data
        else:
            # Assume input is a job title
            if user_input in data['job_title'].values:
                data.loc[data['job_title'] == user_input, 'is_starred'] = 1
            else:
                print("Invalid job title. Please try again.")
                return data

        print(f"\nCandidate '{user_input}' has been starred.")
    except Exception as e:
        print(f"Error: {e}")
    
    return data
star_candidate(data)


Current Candidates:
                                            job_title       fit  rank
0     human resource staffing recruiting professional  0.748748     1
1   retired army national guard recruiter office m...  0.729468     2
2   aspiring human resource professional energetic...  0.725290     3
3   not tech seeking human resource payroll admini...  0.711246     4
4   experienced retail manager aspiring human reso...  0.709471     5
5   human resource coordinator intercontinental no...  0.709192     6
6   aspiring human resource professional passionat...  0.708697     7
7   aspiring human resource manager seeking intern...  0.707899     8
8    ct bauer college business graduate magna cum ...  0.705610     9
9   director human resource north america groupe n...  0.701097    10
10  aspiring human resource manager graduating may...  0.701089    11
11  human resources manager not tech shine north a...  0.700304    12
12  business management major aspiring human resou...  0.698693    13

,job_title,fit,rank,is_starred
0,human resource staffing recruiting professional,0.748748,1,0
1,retired army national guard recruiter office m...,0.729468,2,0
2,aspiring human resource professional energetic...,0.725290,3,0
3,not tech seeking human resource payroll admini...,0.711246,4,0
4,experienced retail manager aspiring human reso...,0.709471,5,0
5,human resource coordinator intercontinental no...,0.709192,6,0
6,aspiring human resource professional passionat...,0.708697,7,0
7,aspiring human resource manager seeking intern...,0.707899,8,0
8,ct bauer college business graduate magna cum ...,0.705610,9,0
9,director human resource north america groupe n...,0.701097,10,0


## Training the LambdaMART Model
- The **LambdaMART algorithm** is used for supervised learning-to-rank tasks.
- Features are engineered from candidate data to represent relevancy scores.
- Supervisory signals (e.g., recruiter "stars") are integrated to refine rankings dynamically.

#### Re-Ranking with Learning-to-Rank

1. **Generate Pairwise Data**: Create pairwise training examples based on starred candidates.

In [ ]:
def generate_pairwise_data(df):
    """Generate pairwise training data for LTR."""
    starred = df[df['is_starred'] == 1]
    not_starred = df[df['is_starred'] == 0]
    
    X, y = [], []
    for _, s_row in starred.iterrows():
        for _, ns_row in not_starred.iterrows():
            # Features: difference of features between starred and not-starred
            X.append([s_row['fit'] - ns_row['fit']])
            y.append(1)  # Positive pair: Starred is better
            
            X.append([ns_row['fit'] - s_row['fit']])
            y.append(0)  # Negative pair: Not-starred is worse
    return np.array(X), np.array(y)

X, y = generate_pairwise_data(data)
SEED = 42
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

print("Pairwise Training Data:")
print(X[:5], y[:5])

2. **Train a LambdaMART Model**: Use `XGBoost` for Learning-to-Rank.

In [78]:
# Train a LambdaMART model (using XGBoost)
model = xgb.XGBRanker(
    objective="rank:pairwise",
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    random_state=42
)

# Group by candidate sets (all candidates in one group for this example)
group = [len(X_train)]

model.fit(X_train, y_train, group=group)

# Predict Scores
data['adjusted_fit'] = model.predict(data[['fit']].values)

3. **Re-Rank Candidates**: Re-rank based on updated scores.

# Re-rank based on adjusted fit scores
data = data.sort_values('adjusted_fit', ascending=False).reset_index(drop=True)
data['rank'] = range(1, len(data) + 1)

print("Re-Ranked Candidates:")
print(data[['candidate_id', 'adjusted_fit', 'rank', 'is_starred']])

### Summary of Steps



1. **Initial Ranking**:
   - Candidates are ranked based on their initial fitness scores (`fit`).

2. **Re-Ranking**:
   - Starred candidates influence the ranking by dynamically updating the model with pairwise preferences.
   - LambdaMART (via XGBoost) is used to learn pairwise rankings, where starred candidates are preferred.

3. **Real-Time Updates**:
   - As new candidates are starred, the pairwise training data is updated, and the model can be fine-tuned or retrained.

In [79]:
# Re-rank based on adjusted fit scores
data = data.sort_values('adjusted_fit', ascending=False).reset_index(drop=True)
data['rank'] = range(1, len(data) + 1)

display(Markdown("### Re-Ranked Candidates:"))
display(data[['rank', 'job_title', 'is_starred',
			  'fit', 'adjusted_fit']])

### Re-Ranked Candidates:

,rank,job_title,is_starred,fit,adjusted_fit
0,1,human resource staffing recruiting professional,0,0.748748,3.622569
1,2,retired army national guard recruiter office m...,0,0.729468,3.622569
2,3,human resources senior specialist,0,0.672054,3.622569
3,4,human resource management major,0,0.671021,3.622569
4,5,aspiring human resource specialist,0,0.670901,3.622569
5,6,human resource generalist schwans,0,0.670881,3.622569
6,7,liberal art major aspiring human resource analyst,0,0.667385,3.622569
7,8,rrp brand portfolio executive not tech japan t...,0,0.666412,3.622569
8,9,people development coordinator not tech,0,0.664587,3.622569
9,10,admission representative community medical cen...,0,0.664097,3.622569


---

### **3. Candidate Filtering**

#### **Approach:**

- Use a 

  preprocessing filter

   to exclude irrelevant candidates before ranking:

  - Text Matching:
    - Use fuzzy matching (Levenshtein distance, cosine similarity) to ensure job_title aligns closely with keywords.
    - Eliminate candidates with low semantic similarity.
  - Location Matching:
    - Set geographical cutoffs if location proximity is essential.
  - Connections:
    - Eliminate candidates with extremely low connections (e.g., <50 if a minimum threshold applies).

#### **Output:**

- A cleaned list of candidates.

#### **Success Metrics:**

- Filter efficiency (percentage of irrelevant candidates removed).
- False positive rate (relevant candidates mistakenly excluded).

In [80]:
from geopy.distance import geodesic
from fuzzywuzzy import fuzz

In [81]:
target_keywords = [
    'aspiring human resources',
    'human resources assistant',
    'hr coordinator',
    'hr generalist (entry-level)',
    'talent acquisition assistant',
    'recruitment coordinator',
    'hr intern',
    'hr trainee',
    'junior hr specialist',
    'hr associate',
    'people operations assistant'
    ]

# User-defined parameters
target_location = "New York"
target_coordinates = (40.7128, -74.0060)  # Latitude and longitude of New York
connection_threshold = 50

In [82]:
def filter_by_text(data, keywords, threshold=60):
    """Filter candidates based on fuzzy text matching with job titles."""
    def calculate_match(job_title):
        scores = [fuzz.partial_ratio(job_title.lower(), keyword.lower()) for keyword in keywords]
        return max(scores)  # Maximum match with any keyword
    
    data['text_match_score'] = data['job_title'].apply(calculate_match)
    return data[data['text_match_score'] >= threshold]


In [83]:
def filter_by_location(data, target_coordinates, max_distance_km=50):
    """Filter candidates based on geographical proximity."""
    def calculate_distance(row):
        candidate_coords = (row['latitude'], row['longitude'])
        return geodesic(candidate_coords, target_coordinates).kilometers
    
    data['distance_to_target'] = data.apply(calculate_distance, axis=1)
    return data[data['distance_to_target'] <= max_distance_km]


In [84]:
def filter_by_connections(data, min_connections):
    """Filter candidates based on minimum connections threshold."""
    return data[data['connections'] >= min_connections]

In [ ]:
# Apply filters sequentially
filtered_data = filter_by_text(data, target_keywords, threshold=60)
filtered_data = filter_by_location(filtered_data, target_coordinates, max_distance_km=50)
filtered_data = filter_by_connections(filtered_data, min_connections=connection_threshold)

print("Filtered Dataset:")
print(filtered_data)


In [ ]:
# Efficiency: Percentage of candidates removed
initial_count = len(data)
filtered_count = len(filtered_data)
efficiency = (initial_count - filtered_count) / initial_count * 100

# Example placeholder for false positives (requires labeled data)
false_positives = 0
false_positive_rate = false_positives / initial_count * 100

print(f"Filter Efficiency: {efficiency:.2f}%")
print(f"False Positive Rate: {false_positive_rate:.2f}%")

Filter Efficiency: 100.00%
False Positive Rate: 0.00%


### **4. Determining a Cut-Off Point for Other Roles**

#### **Approach:**

- Use historical hiring patterns to determine thresholds:
  - Analyze the average fitness score of successful candidates for a role.
  - Use this as a baseline to create cut-offs, dynamically adjustable by user input.
- Incorporate a confidence interval to capture high-potential candidates slightly below the threshold.

#### **Output:**

- Role-specific cut-off scores for ranking.

#### **Success Metrics:**

- Evaluate missed potential hires (false negatives).
- Assess efficiency in reducing manual reviews.